In [1]:
using Distributions

In [2]:
n,p = 100,1000
k = 50
X      = rand(Binomial(2,0.5),n,p)
selQTL = sample(1:p,k,replace=false);
selMrk = setdiff(1:p,selQTL)
Q = X[:,selQTL]
#M = float.(X[:,selMrk])
M = float.(X)
α = randn(k)
α[1] = 100.0
α[2] = 10.0
α[3] =  5.0
a = Q*α
varGen = var(a)
varRes = varGen/4.0
y = a + randn(n)/sqrt(varRes);

In [3]:
varGen

3857.464175456497

In [16]:
using JWAS, DataFrames,CSV,Printf

In [5]:
using Pkg
Pkg.status("JWAS")

    Status `/opt/julia/environments/v1.1/Project.toml`
  [c9a035f4] JWAS v0.5.5 [`~/rohan/Box Sync/JWAS.jl`]


In [6]:
df = DataFrame(ind=1:n,y=y)
rowID = vec(string.(1:n));

In [7]:
modelEQ = "y = intercept"
mme = build_model(modelEQ,varRes)
add_genotypes(mme,M,varGen,header=false,rowID=rowID)

out = runMCMC(mme,df,
    methods = "BayesC",
    Pi = 0.95,
    estimatePi = true,
    chain_length = 50_000,
    burnin = 5000,
    output_samples_frequency=100)

The header (marker IDs) is set to 1,2,...,#markers
1000 markers on 100 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π.
The mean of the prior for the marker effects variance is: 155.082478



A Linear Mixed Model was build using model equations:

y = intercept

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1

MCMC Information:

methods                                      BayesC
                              complete genomic data
                   (i.e., non-single-step analysis)
chain_length                                  50000
burnin                                         5000
estimatePi                                     true
estimateScale                                 false
starting_value                                false
printout_frequency                            50001
output_samples_frequency                        100
constraint   

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:10




The version of Julia and Platform in use:

Julia Version 1.1.0
Commit 80516ca202 (2019-01-21 21:24 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i5-4570 CPU @ 3.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, haswell)
Environment:
  JULIA_DEPOT_PATH = /opt/julia
  JULIA_PKGDIR = /opt/julia
  JULIA_VERSION = 1.1.0


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




Dict{Any,Any} with 8 entries:
  "marker effects"          => 1000×5 DataFrame…
  "Pi"                      => 1×3 DataFrame…
  "heritability"            => 1×3 DataFrame…
  "EBV_y"                   => 100×3 DataFrame…
  "location parameters"     => 1×5 DataFrame…
  "residual variance"       => 1×3 DataFrame…
  "genetic_variance"        => 1×3 DataFrame…
  "marker effects variance" => 1×3 DataFrame…

In [8]:
res = [out["marker effects"][selQTL,[3,4,5]] α]

,Estimate,Std_Error,Model_Frequency,x1
,Any,Any,Any,Float64
1,98.8429,0.985932,1.0,100.0
2,9.54047,0.78769,1.0,10.0
3,5.86846,0.829012,1.0,5.0
4,0.00651465,0.138043,0.00222222,1.41845
5,0.0,0.0,0.0,0.2227
6,0.0,0.0,0.0,-0.561169
7,0.0,0.0,0.0,1.16775
8,0.0,0.0,0.0,0.824239
9,0.0,0.0,0.0,0.0658067


In [9]:
out["marker effects"][selMrk,[3,4,5]]

,Estimate,Std_Error,Model_Frequency
,Any,Any,Any
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.000607068,0.0128636,0.00222222
5,0.0,0.0,0.0
6,0.0,0.0,0.0
7,0.0,0.0,0.0
8,0.0,0.0,0.0
9,0.0,0.0,0.0


In [11]:
CSV.write("phenotypes",df,delim=' ')

"phenotypes"

In [23]:
genFile = open("genotypes","w")

IOStream(<file genotypes>)

In [24]:
# write header for genotype file
print(genFile,"id ")
for i=1:p
    @printf(genFile,"%4d ",i)
end
@printf(genFile,"\n")
# genotypes for i=1:n
for i=1:n
    @printf(genFile,"%4d ",i) # individual id
    for j=1:p
        gen = (M[i,j] - 1.0)*10.0 
        @printf(genFile,"%5.1f ",gen)
    end
    @printf(genFile,"\n")
end

In [25]:
close(genFile)